In [152]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =",device)
gpus = [0]

Device = cuda


In [58]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [59]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([34131372, 1, 55])
Integer dataset size: torch.Size([34131372])


In [60]:
#Get input tensor
def inp(i, shuffle):
    
    #Input (does not include last character in SMILES)
    inp = data[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + seq_length - 1), :, :]

    return inp

In [61]:
#Get target tensor
def target(i, shuffle):
    
    #Target (does not include first character in SMILES)
    target = intdata[int((shuffle[i] * seq_length) + 1) : int((shuffle[i] * seq_length) + seq_length)]

    return target

In [114]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
        self.cuda()  
    
    #Define forward propagation
    def forward(self, inp, hidden):
        output, hidden = self.lstm(inp, hidden)
        output = self.linear(output)
        return output, hidden
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = (Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size)))
        
        return hidden

In [186]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True))

[('Model', 1688), ('Variable', 1016), ('inp', 136), ('mem', 136), ('memory', 136), ('target', 136), ('time_elapsed', 136), ('train', 136), ('F', 80), ('nn', 80), ('np', 80), ('data', 72), ('gpus', 72), ('intdata', 72), ('criterion', 56), ('model', 56), ('optimizer', 56), ('USE_CUDA', 28), ('cuda', 28), ('epochs', 28), ('hidden_size', 28), ('input_size', 28), ('num_layers', 28), ('pytorch_total_params', 28), ('seq_length', 28), ('total_params', 28), ('trainable_params', 28), ('device', 24), ('dropout', 24), ('learning_rate', 24), ('start_time', 24)]


In [180]:
#Set start time
start_time = time.time()

#Define training
def train(epochs):
    
    #Initialize hidden and cell states
    hidden = model.init_states(num_layers, hidden_size)
    
    #Run on GPU if available
    if cuda:
        hidden = (hidden[0].cuda(), hidden[1].cuda())
    
    #Iterate over desired number of epochs 
    for e in range(epochs):
        #Get random order of SMILES molecules (shuffle data)
        shuffle = np.arange(np.shape(data)[0] / seq_length)
        random.shuffle(shuffle)
        
        #Set initial gradients
        model.zero_grad()
    
        #Set initial loss
        loss = 0 
        
        #Iterate over each molecule in dataset
        for i in range(int(np.shape(data)[0] / seq_length)):
            
            #Get input and target
            input_data = inp(i, shuffle).float()
            target_data = target(i, shuffle).long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward(retain_graph=True)
            optimizer.step()
            
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.8f}".format(loss.data.item() / seq_length) + " | Epoch: " + str(e) + " | Iteration: " + str(i) + " | Time elapsed: " + str(hours) + " hours " + str(minutes) + " minutes " + str(seconds) + " seconds ")
        

In [181]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 256
num_layers = 3
dropout = .2
learning_rate = 0.001
epochs = 10
seq_length = 76

In [182]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [183]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 1387319


In [185]:
#Train
train(epochs)

Loss: 0.04890399 | Epoch: 0 | Iteration: 0 | Time elapsed: 0 hours 1 minutes 43 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 0.08845430 | Epoch: 0 | Iteration: 1 | Time elapsed: 0 hours 1 minutes 43 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 0.13589836 | Epoch: 0 | Iteration: 2 | Time elapsed: 0 hours 1 minutes 43 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 0.20221474 | Epoch: 0 | Iteration: 3 | Time elapsed: 0 hours 1 minutes 43 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16)

Loss: 1.68797824 | Epoch: 0 | Iteration: 30 | Time elapsed: 0 hours 1 minutes 57 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 1.75919442 | Epoch: 0 | Iteration: 31 | Time elapsed: 0 hours 1 minutes 58 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 1.82111037 | Epoch: 0 | Iteration: 32 | Time elapsed: 0 hours 1 minutes 59 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 1.87933852 | Epoch: 0 | Iteration: 33 | Time elapsed: 0 hours 2 minutes 0 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 

Loss: 5.01786724 | Epoch: 0 | Iteration: 60 | Time elapsed: 0 hours 2 minutes 36 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 5.23058078 | Epoch: 0 | Iteration: 61 | Time elapsed: 0 hours 2 minutes 38 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 5.39628079 | Epoch: 0 | Iteration: 62 | Time elapsed: 0 hours 2 minutes 40 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 5.52310422 | Epoch: 0 | Iteration: 63 | Time elapsed: 0 hours 2 minutes 42 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i',

Loss: 11.82074697 | Epoch: 0 | Iteration: 90 | Time elapsed: 0 hours 3 minutes 41 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 12.02214050 | Epoch: 0 | Iteration: 91 | Time elapsed: 0 hours 3 minutes 43 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 12.28910587 | Epoch: 0 | Iteration: 92 | Time elapsed: 0 hours 3 minutes 46 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), ('i', 16), ('input_data', 16), ('loss', 16), ('minutes', 16), ('output', 16), ('seconds', 16), ('shuffle', 16), ('target_data', 16)]
Loss: 12.49677156 | Epoch: 0 | Iteration: 93 | Time elapsed: 0 hours 3 minutes 48 seconds 
[('epochs', 28), ('e', 16), ('hidden', 16), ('hours', 16), (

KeyboardInterrupt: 